In [134]:
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/MSGC_DATA/Tree_Spec_Lib/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

In [135]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom = NULL, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: PLS-LDA (plsgenomics)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    #print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    #print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [136]:
add_model_to_manifest <- function(
    model_id, 
    outlier = "", 
    preprocessing="",
    source="", 
    weight = "",
    n = "",
    oob_error = "",
    accuracy = "",
    r2 = "",
    chi2prob = "",
    logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,weight,n,oob,accuracy,r2,rpd,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        source,
        outlier,
        preprocessing,
        weight,
        n,
        oob_error,
        accuracy,
        r2,
        chi2prob,
        sep=","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [137]:
train_pls_lda <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    ncomp = 32,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    save_path = "./mle/models",
    seed = NULL,
    log_string = ""
){

    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    #x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        #x_test <- preprocess_fn(x_test)
    }

    sample_perm <- permute::shuffle(length(train_labels))
    sample_index <- create_stratified_sample(
        train_labels,
        permutation = sample_perm,
        samples_per_pft = 300
    )
    x_train <- x_train[sample_perm,][sample_index,]
    train_labels <- train_labels[sample_perm][sample_index]

    #if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
        levels(test_labels) <- c(levels(test_labels), "Forb")
    #    }
    
    model_s3_obj <- list(
        x_train = x_train,
        y_train = train_labels,
        ncomp = ncomp,
        nruncv = 0
    )

    class(model_s3_obj) <- "pls_lda"

    save(
        model_s3_obj,
        file = file.path(save_path, paste0(model_id, ".rda"))
    )

    # create predictions (ranger)
    model_predictions <- predict(
            model_s3_obj, 
            x_test
        ) 

    print(model_predictions)


    # generate the confusion matrix

    confusion_matrix <- caret::confusionMatrix(
        model_predictions %>% as.factor(),
        test_labels,
        mode = "everything"
    )

    log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_rf_n.log")

    return(
        list(
                model = model_s3_obj,
                confusion = confusion_matrix %>% as.list()
                )
    )

    

}

In [138]:
apply_model.pls_lda <- function(x, model,  ...){
    
    if("x" %in% colnames(x) && "y" %in% colnames(x)){
        print("Spatial ifnormation detected, ignoring....")
        target_df <- subset(x, select=-c(x,y))
        prediction_df <- predict.pls_lda(model, target_df) %>% as.data.frame()
        prediction_df$x <- x$x
        prediction_df$y <- x$y

    } else {
        prediction_df <- predict.pls_lda(model, x) %>% as.data.frame()
    }

    return(prediction_df)
}

In [139]:
write_pls_lda_model <- function(object, save_path, uuid = NULL){
    model_id <- uuid
    if(is.null(model_id)){
        model_id <- uuid::UUIDgenerate()
    }

    save(x_train, x_train_path)
}

In [140]:
predict.pls_lda <- function(pls_lda, x, ...){

    predctions <- plsgenomics::pls.lda(
        pls_lda$x_train,
        pls_lda$y_train %>% as.numeric() + 1,
        as.data.frame(x),
        pls_lda$ncomp,
        pls_lda$nruncv
    )$predclass

    predictions_df <- as.data.frame(as.numeric(predctions) - 1)
    colnames(predictions_df) <- c("z")
    predictions_df$x <- rep(0,nrow(predictions_df))
    predictions_df$y <- rep(0,nrow(predictions_df))


    return(convert_fg1_int( predictions_df)$z %>% as.factor())
    
}

In [141]:
filter_target_subset <- function(df){

    df_cols <- colnames(df) %>% as.character()
    target_cols <- c(
        "Vogelmann",
        "TVI",
        "CRI4",
        "Datt5",
        "DWSI4",
        "GDVI",
        "MCARI",
        "MTVI",
        "NPCI",
        "PARS",
        "X417.593_5nm",
        "X787.593_5nm",
        "X892.593_5nm"
    )

    used_cols <- intersect(df_cols, target_cols)
    print(paste0("Filtering Data frame to the following columns: ", used_cols))
    return(df[,used_cols])
}

In [142]:
read_pls_lda_model <- function(pls_lda_obj){
    x_train

}

In [154]:
train_model <- function(
    train_df, 
    train_labels,
    #test_df, 
    #test_labels,
    ntree = 50,
    max_depth = NULL,
    mtry = NULL,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = NULL,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    seed = NULL,
    log_string = ""
){
    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    #x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        #x_test <- preprocess_fn(x_test)
    }

    sample_perm <- permute::shuffle(length(train_labels))
    #sample_index <- create_stratified_sample(
    #    train_labels,
    #    permutation = sample_perm,
    #    samples_per_pft = 20
    #)
    #x_train <- x_train[sample_perm,][sample_index,]
    #train_labels <- train_labels[sample_perm][sample_index]

    model <- ranger::ranger(
            num.trees = ntree,
            mtry = mtry,
            max.depth = max_depth,
            #case.weights = weight_fn(train_labels),
            classification = TRUE,
            x=x_train,
            y=train_labels
        )

    #if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
    #        levels(test_labels) <- c(levels(test_labels), "Forb")
    #        }

    # create predictions (ranger)
        #model_predictions <- predict(
        #    model, 
        #    x_test
        #)$prediction %>% as.factor()

        # generate the confusion matrix

        #confusion_matrix <- caret::confusionMatrix(
        #    model_predictions, 
        #    test_labels,
        #    mode = "everything"
        #)

        # generate an id to uniquely identify the model
        #model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        log_model_results(
            model_id = model_id,
            #confusion_matrix = confusion_matrix,
            custom = log_string,
            #distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_pls_lda.log")

        # track what levels are associated with the UUID

        # save the model using the model UUID
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))
        
        return(
            #list(
                model #= model,
                #confusion = confusion_matrix %>% as.list()
                #)
        )
}

In [155]:
base_paths <- c(
    #"img_raw_raw.csv",
    #"corrected_and_img.csv",
    "grd_raw_raw.csv"#,
    #"bison_gulch_stratified.csv",
    #"grd_raw_corrected.csv",
    #"img_indices_only.csv",# include veg indices
    #"grd_indices_only.csv"
)

In [156]:
calculate_posterior_weights <- function(validation_path ="figures/merged_validation_s.csv" ){

    validation_df <- read.csv(validation_path, header = TRUE)
    #print(head(validation_df))

    total_observations <- sum(validation_df$validation_counts)
    #print(total_observations)
    weights <- (1/ validation_df$validation_prop)
    #print(validation_df$validation_prop)

    total_by_fg1 <- aggregate(
        x = validation_df$validation_counts,
        by = list(validation_df$key),
        FUN = sum
    )

    fg1_weight_list <- list()

    for( row_idx in seq(nrow(total_by_fg1))){
        name <- total_by_fg1$Group.1[[row_idx]]
        value <- total_by_fg1$x[[row_idx]]
        fg1_weight_list[name] <- value
    }
    
    return(fg1_weight_list)
}

get_posterior_weights_from_targets <- function(target_factor, posterior_weight = calculate_posterior_weights()){
    unbiased_weights <- targets_to_weights(target_factor)

    target_name_char <- target_factor %>% as.character()

    output_weights <- seq_along(target_factor)

    for(i in seq_along(target_factor)){
        if(posterior_weight[[target_name_char[[i]]]] > 0){
            fg1_weight <- 1 / posterior_weight[[target_name_char[[i]]]]
        } else {
            fg1_weight <- 0
        }
        output_weights[[i]] <- unbiased_weights[[i]] * fg1_weight
    }

    return(output_weights)
}

## Hypotheses

Peter
* Increasing number of trees will improve the accuracy/validation mismatch
* bias in 

Ken
* decreasing the number of the trees will decrease the impact of the posterior weighting on chi-squared statistic

In [157]:
outlier_functions <- list(
    #clip = load_model("./mle/clip_transform.rda"),
    no_treatment = function(x, ignore_cols = NULL){return(x)}# no transform
)

outlier_treatments <- c(
    "no_treatment"#,
    #"clip"
)

preprocess_functions <- list(
    no_treatment = function(x, ignore_cols = NULL){return(x)},# no transform
    min_max = columnwise_min_max_scale,
    robust = columnwise_robust_scale,
    standard = standardize_df
)

weight_functions <- list(
    posterior = get_posterior_weights_from_targets,
    balanced = targets_to_weights,
    no_treatment = function(x){return(NULL)}# No weights
)
weight_treatments <- c(
    "no_treatment"#,
    #"balanced"#,
    #"posterior"
)

preprocessing_treatments <- c(
    "no_treatment"#,
    #"min_max",
    #"standard",
    #"robust"
)

In [175]:
num_components <- c(
    #1,
    #2,
    ##4,6,
    #8,#
    #10,
    #12,#14,
    #16,#18,20,24,
    #32,#50,
    #64,#100,
    #128,
    #256,
    #512#,1024
    500,
    1000,
    5000
)

depth_control <- c(
    #10,20, 
    45
)

In [176]:
to_fg0 <- function(target_factor){
    return(
        change_aggregation(
            target_factor,
            0,
            rjson::fromJSON(file="./assets/pft_adj_list.json")
        )
    )
}

In [177]:
#test_data <- subset(read.csv("Data/gs/x_test/img_raw_raw.csv"), select = -c(X))
#test_labels <- read.csv("Data/gs/y_test/img_raw_raw.csv")$x %>% to_fg0() %>% as.factor()
#train_labels <- read.csv("Data/gs/y_train/img_raw_raw.csv")$x %>% as.factor()
train_labels <- read.csv("Data/gs/y_train/grd_raw_raw.csv")$x %>% as.factor()

#manifest_path <- "./gs_manifest_reduced_val.csv"

In [178]:
train_labels

[1] abibal abibal abibal abibal abibal abibal abibal abibal abibal abibal
 [11] abibal abibal abibal abibal abibal abibal abibal abibal abibal abibal
 [21] abibal abibal abibal abibal abibal abibal abibal abibal abibal abibal
 [31] abibal abibal abibal abibal abibal abibal abibal abibal abibal abibal
 [41] abibal abibal abibal abibal abibal abibal abibal abibal abibal abibal
 [51] abibal abibal abibal abibal abibal abibal abibal abibal abibal abibal
 [61] abibal abibal abibal abibal abibal abibal abibal abibal acepen acepen
 [71] acepen acepen acepen acepen acepen acepen acepen acepen acepen acepen
 [81] acepen acepen acepen acepen acepen acepen acepen acepen acepen acepen
 [91] acepen acepen acepen acepen acepen acepen acepen acepen acepen acepen
[101] acepen acepen acepen acepen acepen acerub acerub acerub acerub acerub
[111] acerub acerub acerub acerub acerub acerub acerub acerub acerub acerub
[121] acerub acerub acerub acerub acerub acerub acerub acerub acerub acerub
[131] acerub acerub acerub acerub acerub acerub acerub acerub acerub acerub
[141] acerub acerub acerub acerub acerub acerub acerub acerub acerub acerub
[151] acerub acerub acerub acerub acerub acerub acerub acerub acerub acesac
[161] acesac acesac acesac acesac acesac acesac acesac acesac acesac acesac
[171] alninc alninc alninc alninc alninc alninc alninc alninc alninc alninc
[181] alninc alninc alninc alninc alninc alninc alninc alninc alninc alninc
[191] alninc alninc alninc alninc alninc alninc alninc betall betall betall
[201] betall betall betall betall betall betall betall betall betall betall
[211] betall betall betall betall betall betall betall betall betall betall
[221] betall betall betall betall betall betall betall betall betall betall
[231] betall betall betall betall betall betpap betpap betpap betpap betpap
[241] betpap betpap betpap betpap betpap betpap betpap betpap betpap betpap
[251] betpap betpap betpap betpap betpap betpap betpap betpap betpap betpop
[261] betpop betpop betpop betpop betpop betpop betpop betpop betpop betpop
[271] betpop betpop betpop betpop betpop betpop betpop betpop betpop betpop
[281] betpop betpop betpop betpop betpop betpop betpop betpop betpop betpop
[291] betpop betpop betpop faggra faggra faggra faggra faggra faggra faggra
[301] faggra faggra faggra faggra faggra faggra faggra faggra faggra faggra
[311] faggra faggra faggra faggra faggra faggra faggra faggra faggra faggra
[321] faggra faggra faggra faggra faggra faggra faggra faggra faggra faggra
[331] faggra faggra faggra faggra faggra faggra faggra faggra faggra faggra
[341] fraame fraame fraame fraame fraame fraame fraame fraame fraame fraame
[351] fraame fraame fraame fraame fraame fraame fraame fraame fraame fraame
[361] fraame fraame fraame fraame fraame fraame larlar larlar larlar larlar
[371] larlar larlar larlar larlar larlar larlar larlar larlar larlar larlar
[381] larlar larlar larlar larlar larlar larlar picgla picgla picgla picgla
[391] picgla picgla picgla picgla picgla picgla picgla picgla picgla picgla
[401] picgla picgla picgla picgla picgla picgla picgla picgla picgla picgla
[411] picgla picgla picgla picgla picgla picgla picgla picgla picgla picgla
[421] picgla picgla picgla picgla picrub picrub picrub picrub picrub picrub
[431] picrub picrub picrub picrub picrub picrub picrub picrub picrub picrub
[441] picrub picrub picrub picrub picrub picrub picrub picrub picrub picrub
[451] picrub picrub picrub picrub picrub picrub picrub picrub picrub picrub
[461] picrub picrub picrub picrub picrub picrub picrub picrub picrub picrub
[471] picrub picrub picrub picrub picrub picrub picrub picrub picrub picrub
[481] picrub picrub picrub picrub picrub picrub pinstr pinstr pinstr pinstr
[491] pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr
[501] pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr
[511] pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr pinstr
[521] pinstr pinstr pinstr pinstr pinstr pinstr pi

In [179]:
rf_model_results <- train_model(
    train_data,
    train_labels, 
    #test_data,
    #test_labels,
    ntree = 100,
    max_depth = NULL,#d,
    #outlier_fn = outlier_functions[[o_treatment]],
    #preprocess_fn = preprocess_functions[[p_treatment]],
    #weight_fn = weight_functions[[w_treatment]],
    #model_id = model_id,
    seed=61718#,
    #log_string = paste(n, filepath, o_treatment, p_treatment, w_treatment)
)
#print(rf_model_results)
rf_model <- rf_model_results$model

In [180]:
for(filepath in base_paths){
    train_data <- subset(read.csv(paste0("Data/gs/x_train/", filepath)), select = -c(X))
    labels <- read.csv(paste0("Data/gs/y_train/", filepath))$x  %>% as.factor() #to_fg0() 
    for(o_treatment in outlier_treatments){
        for(p_treatment in preprocessing_treatments){
            for(w_treatment in weight_treatments){
                for(n in num_components){
#                    for(d in depth_control){

                model_id <- uuid::UUIDgenerate()
                save_path <- paste0("mle/experiments/gs/", model_id, "/")
                if(!dir.exists(save_path)){
                    dir.create(save_path)
                }

                rf_model_results <- train_model(
                    #filter_df_bands(train_data), 
                    train_data,
                    labels, 
                    #test_data,
                    #test_labels,
                    ntree = n,
                    max_depth = NULL,#d,
                    outlier_fn = outlier_functions[[o_treatment]],
                    preprocess_fn = preprocess_functions[[p_treatment]],
                    weight_fn = weight_functions[[w_treatment]],
                    model_id = model_id,
                    seed=61718,
                    log_string = paste(n, filepath, o_treatment, p_treatment, w_treatment)
                )
                #print(rf_model_results)

                rf_model <- rf_model_results$model
                #acc <- as.list(rf_model_results$confusion$overall)$Accuracy
                #print(acc)

                #if(acc > 0.6){


                #results <- validate_model(
                #    rf_model, 
                #    save_path, 
                #    outlier_processing = outlier_functions[[o_treatment]],
                #    transform_type = preprocess_functions[[p_treatment]],
                #    pft_aggregation = 0
                #)

                #aggregated_results <- aggregate_results(save_path)

                # calculate validation statistics
                #chi2 <- calculate_chi_squared_probability(aggregated_results)
                #r2 <- calculate_validation_r2(aggregated_results)
                #rpd <- calculate_rpd(aggregated_results)
#
                #add_model_to_manifest(
                #    model_id = model_id,
                #    outlier = o_treatment,
                #    preprocessing = p_treatment,
                #    source = filepath,
                #    weight = w_treatment,
                #    n = n,
                #    oob_error = rf_model$prediction.error,
                #    accuracy = acc,
                #    r2 = r2,
                #    chi2prob = rpd,
                #    logpath=manifest_path
                #)
#
                #plot_by_pft(
                #    aggregated_results,
                #    save_path = paste0(save_path, "aggregate.html"),
                #    open = FALSE,
                #    image_path = NULL,
                #    aggregation=0
                #)
            ##
                #write_validation_table(
                #    aggregated_results,
                #    save_path = paste0(save_path, "table.html"),
                #    open = FALSE
                #)
                #} else {
                #    add_model_to_manifest(
                #        model_id = model_id,
                #        outlier = o_treatment,
                #        preprocessing = p_treatment,
                #        source = filepath,
                #        weight = w_treatment,
                #        n = n,
                #        oob_error = rf_model$prediction.error,
                #        accuracy = acc,
                #        r2 = "Skipped",
                #        chi2prob = "Skipped",
                #        logpath=manifest_path
                #    )

                #}
                   # }
                }
            }
        }
    }
}

In [183]:
#list.files("mle/models/gs")
mod<-load_model("mle/models/gs/62f07a30-dd1e-4621-8752-abafe2803378.rda")
mod$confusion.matrix

        predicted
true     abibal acepen acerub acesac alninc betall betpap betpop faggra fraame
  abibal     34      0      1      0      1      0      0      2      0      0
  acepen      0     29      1      0      4      1      0      0      1      0
  acerub      0      0     37      0      0      3      1      0      5      1
  acesac      0      0      0      4      0      2      3      0      1      0
  alninc      0      5      2      0     14      0      0      0      1      1
  betall      0      2      2      3      0     24      0      0      0      2
  betpap      0      0      2      1      0      1     12      2      2      0
  betpop      0      0      2      0      0      1      4     15      5      0
  faggra      0      2      3      0      0      0      0      2     37      1
  fraame      0      0      0      0      0      1      0      2      1     20
  larlar      4      0      0      0      0      0      0      0      0      0
  picgla      3      0      0     